# Flight Planning Domain 

This notebook aims to make a short and interactive example of the Flight Planning Domain. You can find more information about it in the README file.

In [ ]:
from skdecide.hub.domain.flight_planning.domain import FlightPlanningDomain, WeatherDate
from skdecide.hub.domain.flight_planning.weather_interpolator.weather_tools.get_weather_noaa import (
    get_weather_matrix,
)
from skdecide.hub.domain.flight_planning.weather_interpolator.weather_tools.interpolator.GenericInterpolator import (
    GenericWindInterpolator,
)

### Definition of the problem

Here we will make a short haul flight from Paris Charles de Gaulle airport (ICAO : LFPG) to Toulouse-Blagnac airport (ICAO: LFBO), using an airbus A320 aircraft.

In [ ]:
origin = "LFPG"
destination = "LFBO"
aircraft = "A320"

Now, we are going to define a date, in the past 6 months, to get access to weather interpolation.

In [ ]:
weather_date = WeatherDate(day=21, month=4, year=2023)
w_dict = weather_date.to_dict()
mat = get_weather_matrix(
    year=w_dict["year"],
    month=w_dict["month"],
    day=w_dict["day"],
    forecast=w_dict["forecast"],
    delete_npz_from_local=False,
    delete_grib_from_local=False,
)
wind_interpolator = GenericWindInterpolator(file_npz=mat)

We can now define heuristic & cost function, to feed the A* solver. This aims to guide him along the airways graph to solve the problem, and get as close as possible to the optimal solution. 

The heuristic function can be either fuel, time, distance, lazy_fuel, lazy_time and None. If None, the A* will behave as a Dijkstra like search algorithm, as we give a 0 value to the A* algorithm. 

The cost function can be either fuel, time and distance. It will define the cost of the flight plan, computed during the state to state flight simulation. 

In [ ]:
heuristic = "lazy_fuel"
cost_function = "fuel"

### Definition of the Domain and Optional features 

There is many other optionnal features, as described in the README. We are now going to define the domain. 

It will initialize the Flight Planning Domain with the given features, and can take some time, especially if it needs to download the differents weather files, and if you ask for a fuel loop. 


In [ ]:
domain_factory = lambda: FlightPlanningDomain(
    origin,
    destination,
    aircraft,
    weather_date=weather_date,
    wind_interpolator=wind_interpolator,
    heuristic_name=heuristic,
    objective=cost_function,
    fuel_loop=False,
    graph_width="normal",
)
domain = domain_factory()

Then we can solve the problem : (can take time depending on )

In [ ]:
domain.solve(domain_factory, make_img=True)